Подключаем библиотеки

In [1]:
import pandas as pd

Загружаем нужные файлов для работы

In [2]:
events = pd.read_csv('gd3/events.csv', sep=',')
events_df = events.copy()

purchase = pd.read_csv('gd3/purchase.csv', sep=',')
purchase_df = purchase.copy()

Из всего массива данных для каждого файла отбираем данные за 2018 год и проводим преобразование типов, где это необходимо

In [3]:
cond = (events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type=='registration')
registered = events_df[cond]['user_id'].to_list()
events_2018 = events_df[events_df.user_id.isin(registered)]
events_2018.start_time = pd.to_datetime(events_2018.start_time, format='%Y-%m-%dT%H:%M:%S')
events_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              66959 non-null  int64         
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         66959 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


C:\Users\nason\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [4]:
cond2 = (purchase_df.event_datetime>='2018-01-01') & (purchase_df.event_datetime<'2019-01-01')
registered2 = purchase_df[cond2]['user_id'].to_list()
purchase_2018 = purchase_df[purchase_df.user_id.isin(registered)]
purchase_2018.event_datetime = pd.to_datetime(purchase_2018.event_datetime, format='%Y-%m-%dT%H:%M:%S')
purchase_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1600 non-null   int64         
 1   user_id         1600 non-null   int64         
 2   event_datetime  1600 non-null   datetime64[ns]
 3   amount          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


In [5]:
events_2018.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
51405,80308,registration,NaN,2018-01-01 03:48:40,NaN,27832
51406,80309,registration,NaN,2018-01-01 04:07:25,NaN,27833
51407,80310,registration,NaN,2018-01-01 08:35:10,NaN,27834
51408,80311,registration,NaN,2018-01-01 11:54:47,NaN,27835
51409,80312,registration,NaN,2018-01-01 13:28:07,NaN,27836


In [6]:
purchase_2018.head()

,id,user_id,event_datetime,amount
1171,16845,27845,2018-01-03 18:53:43,100
1172,16846,27865,2018-01-04 14:46:10,250
1174,16848,27911,2018-01-07 08:19:12,50
1175,16849,27910,2018-01-07 12:11:34,100
1176,16850,27940,2018-01-07 13:16:41,200


Переименуем колонки с одинаковыми именами (id) и сделаем объединенный датафрейм

In [7]:
events_2018 = events_2018.rename(columns={"id": "event_id"})
purchase_2018 = purchase_2018.rename(columns={"id": "purchase_id"})
total_events_2018 = pd.concat([events_2018,purchase_2018],sort=False)
total_events_2018.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN


Определим существующие типы сложностей, которые можно выбрать

In [8]:
total_events_2018['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

Теперь, видя какие существуют типы сложности, создадим группы группы пользователей в зависимости от выбранной сложности

In [9]:
users_with_easy_level_2018 = total_events_2018[total_events_2018['selected_level'] == 'easy']['user_id'].unique()
users_with_medium_level_2018 = total_events_2018[total_events_2018['selected_level'] == 'medium']['user_id'].unique()
users_with_hard_level_2018 = total_events_2018[total_events_2018['selected_level'] == 'hard']['user_id'].unique()

Определим для каждой группы процент пользователей, оплативших пакеты вопросов

In [11]:
user_groups_2018 = [
    {'easy': users_with_easy_level_2018},
    {'medium': users_with_medium_level_2018},
    {'hard': users_with_hard_level_2018}
]

for group in user_groups_2018:
    selected_level = list(group.keys())[0]
    group_users = group[selected_level]
    count_of_users_in_group = len(group_users)
    purchase_2018_slice = purchase_2018[purchase_2018['user_id'].isin(group_users)]
    percent_of_purchase = purchase_2018_slice['user_id'].nunique()/count_of_users_in_group
    print ('Процент пользователей, оплативших пакеты вопросов и выбравших уровень сложности {}: {:.2%}'.format(selected_level,percent_of_purchase))

Процент пользователей, оплативших пакеты вопросов и выбравших уровень сложности easy: 7.72%
Процент пользователей, оплативших пакеты вопросов и выбравших уровень сложности medium: 20.86%
Процент пользователей, оплативших пакеты вопросов и выбравших уровень сложности hard: 35.39%


Определим для каждой группы среднее время между событиями оплаты и выбором уровня сложности

In [12]:
user_groups_2018 = [
    {'easy': users_with_easy_level_2018},
    {'medium': users_with_medium_level_2018},
    {'hard': users_with_hard_level_2018}
]

for group in user_groups_2018:
    level = list(group.keys())[0]
    group_users = group[level]
    purchase_2018_slice = purchase_2018[purchase_2018['user_id'].isin(group_users)]
    level_choice_2018 = total_events_2018[(total_events_2018['event_type'] == 'level_choice') & (total_events_2018['user_id'].isin(group_users))]
    if (level_choice_2018['user_id'].value_counts().mean()) == 1:
        level_choice_2018 = level_choice_2018[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
        purchase_2018_slice_2 = purchase_2018_slice[['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})
        merged_2018 = purchase_2018_slice_2.merge(level_choice_2018,on='user_id',how='inner')
        merged_2018['timedelta'] = merged_2018['purchase_time'] - merged_2018['level_choice_time']
        mean_time = merged_2018['timedelta'].mean()
        print ('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности {}: {}'.format(level,mean_time))

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности easy: 3 days 14:58:52.941798941
Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности medium: 3 days 23:14:13.165118679
Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности hard: 3 days 07:20:41.420814479


Определим существует ли разница во времени между событиями регистрации и оплаты для групп пользователей с разным уровнем сложности

In [13]:
user_groups_2018 = [
    {'easy': users_with_easy_level_2018},
    {'medium': users_with_medium_level_2018},
    {'hard': users_with_hard_level_2018}
]

for group in user_groups_2018:
    level = list(group.keys())[0]
    group_users = group[level]
    purchase_2018_slice = purchase_2018[purchase_2018['user_id'].isin(group_users)]
    registration_2018 = total_events_2018[(total_events_2018['event_type'] == 'registration') & (total_events_2018['user_id'].isin(group_users))]
    if (registration_2018['user_id'].value_counts().mean()) == 1:
        registration_2018 = registration_2018[['user_id','start_time']].rename(columns={'start_time':'registration_time'})
        purchase_2018_slice_2 = purchase_2018_slice[['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})
        merged_df = purchase_2018_slice_2.merge(registration_2018,on='user_id',how='inner')
        merged_df['timedelta'] = merged_df['purchase_time'] - merged_df['registration_time']
        mean_time = merged_df['timedelta'].mean()
        print ('Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности {}: {}'.format(level,mean_time))

Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности easy: 3 days 22:10:23.211640211
Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности medium: 4 days 06:12:06.576883384
Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности hard: 3 days 14:55:19.257918552


### Выводы

Пользователей оплативших пакеты вопросов тем больше, чем сложнее уровень они выбрали.

Процент пользователей, оплативших пакеты вопросов и выбравших уровень сложности easy: 7.72%. 
Процент пользователей, оплативших пакеты вопросов и выбравших уровень сложности medium: 20.86%. 
Процент пользователей, оплативших пакеты вопросов и выбравших уровень сложности hard: 35.39%. 


Степень вероятности оплаты от выбранного пользователем уровня сложности прослеживается такая.
Пользователи, выбравшие уровень сложности hard потратили наименьшее среднее время на принятие решения о покупке. Оно составило 3 дня 7 часов 20 минут.
Для пользователей, выбравших  уровень сложности easy на принятие решения о покупке потратили в среднем 3 дня 14 часов 58 минут.
Медленнее всех определялись с покупкой пользователи, выбравшие уровень сложности medium. Их среднее время составило 3 дня 23 часа 14 минут.


Похожая тенденция наблюдается и между временными событиями регистрации и оплаты для групп пользователей с разным уровнем сложности.
Пользователи, выбравшие уровень сложности hard потратили наименьшее среднее время на принятие решения о покупке после регистрации. Оно составило 3 дня 14 часов 55 минут.
Пользователи, выбравших  уровень сложности easy на принятие решения о покупке после регистрации потратили в среднем 3 дня 22 часа 10 минут.
И пользователи, выбравших  уровень сложности medium на принятие решения о покупке после регистрации потратили в среднем 4 дня 6 часов 12 минут.